감성 분석 - https://mindscale.kr/course/python-text-classification/multinomial/

문서 분류 - https://mindscale.kr/course/python-text-classification/doc-class-preprocess/

## 감성분석

감성분석이란 문서에 나타난 긍부정의 감정 또는 태도를 분석하는 것입니다. 감정 분석에는 크게 사전에 의한 방법과 기계 학습에 의한 방법 2가지가 있다.

사전에 의한 방법은 단어 중에 긍/부정의 감성을 나타내는 단어들을 사전으로 만들고, 문서에서 긍정 단어, 부정 단어의 비율을 계산해서 긍정 단어가 많으면 긍정으로, 부정 단어가 많으면 부정으로 판정하는 것. 감성 사전을 만들기 위해서는 높은 전문성과 많은 노력이 필요하다는 단점이 있다.

기계학습에 의한 방법은 미리 긍/부정으로 분류된 문서들을 수집하여 기계학습 모형에 학습시키는 방법. 여기서는 아마존 핸드폰 리뷰 데이터를 기반으로 기계학습으로 감성 분석을 실시하는 방법을 알아보기로 한다.

#### 데이터 준비

In [1]:
import requests
res = requests.get('https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip')
with open('sentiment labelled sentences.zip', 'wb') as f:
    f.write(res.content)

In [2]:
from zipfile import ZipFile
z = ZipFile('sentiment labelled sentences.zip')
data = z.open('sentiment labelled sentences/amazon_cells_labelled.txt')

In [3]:
import pandas as pd
df = pd.read_csv(data, sep="\t", header=None)
df.head()

# 0번 컬럼은 리뷰, 1번 컬럼에는 부정적 리뷰가 0, 긍정적 리뷰가 1

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


#### 단어문서 행렬 만들기

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=1000, stop_words='english')
tdm = cv.fit_transform(df[0])

#### 데이터 분할

In [7]:
x = tdm
y = df[1]

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train

<800x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 3220 stored elements in Compressed Sparse Row format>

#### 선형 모형을 통한 분류

In [ ]:
import tensorflow as tf

* Dense(1): 1개의 예측(긍/부정)을 하는 선형 모형을 만듭니다.
* input_shape=(1000,): 1000개의 단어를 입력 받습니다.
* activation=: 시그모이드 활성화 함수를 사용합니다.

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1, input_shape=(1000,), activation='sigmoid'))

* 학습 알고리즘(optimizer)는 adam을 사용합니다.
* 손실함수(loss)는 교차 엔트로피(binary_crossentropy)를 사용합니다.
* 보조적인 지표로 정확도(accurary)를 사용합니다. 정확도란 전체 사례 중 맞은 사례의 비율을 의미합니다.

In [ ]:
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=10)

In [ ]:
model.evaluate(x_test, y_test) # 모형 평가

#### 가중치

In [ ]:
weight, bias = model.trainable_weights

In [ ]:
import pandas as pd

word_weight = pd.DataFrame({
    '단어': cv.get_feature_names(),
    '가중치': weight.numpy().flat
})

In [ ]:
word_weight.sort_values('가중치').head()

In [ ]:
word_weight.sort_values('가중치', ascending=False).head()

## 문서 분류

#### 데이터 준비

In [16]:
from sklearn.datasets import fetch_20newsgroups

categories = [
    'rec.motorcycles',
    'rec.sport.baseball',
    'rec.sport.hockey'
]

newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)

#### TF-IDF로 단어 문서 행렬 만들기

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', max_features=1000)
x_train = tfidf.fit_transform(newsgroups_train.data)
x_train.shape

(1795, 1000)

In [ ]:
x_test = tfidf.transform(newsgroups_test.data)
# 이때는 x_train과 똑같은 방식으로 단어 문서 행렬을 만들어야 하므로 fit_transform 대신에 transform을 사용.
y_train = newsgroups_train.target
y_test = newsgroups_test.target

#### 다항 분류 모형

텐서플로를 사용하여 다항 분류 모형을 만든다.

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(
    units=3,
    input_shape=(1000,),
    kernel_regularizer=tf.keras.regularizers.l1_l2(0.003, 0.005),
    activation='softmax'))

* tf.keras.layers.Dense() 로 층(완전 연결층)을 추가합니다. 세부 옵션은 다음과 같습니다.
* units: 최종 출력의 개수를 유닛(unit)이라 합니다. 본 예시의 목표는 3가지 다항 분류이므로 최종 출력의 개수가 3 이 됩니다.
* input_shape: 입력데이터의 크기 설정. 앞서 max_feature 개수를 1000으로 지정하였으므로 (1000,) 과 같이 튜플 형태로 입력합니다.
* kernel_regularizer: 단어들의 가중치가 지나치게 커지지 않도록 정규화항을 추가해줍니다. l1_l2는 L1과 L2, 2가지 정규화를 모두 해줍니다. L1은 가중치의 절대값을 손실함수에 추가합니다. L2는 가중치의 제곱을 손실함수에 추가합니다. 어느 쪽이든 가중치가 커지면 손실이 커지므로, 가중치가 지나치게 커지는 것을 막아줍니다. l1_l2(0.003, 0.005)는 손실함수에서 L1항에 0.003을 곱하고 L2항 0.005를 곱하라는 뜻입니다. 여기서 0.003, 0.005와 같은 수치는 다양하게 바꿔보면서 성능이 가장 잘 나오는 수치를 선택합니다.
* activation: 활성화 함수로 소프트맥스 함수를 사용합니다.

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])

* loss: 모형의 손실함수는 sparse_categorical_crossentropy를 사용
* optimizer: 여기에서 최적화 함수는 Adam (Adaptive Moment Estimation: 적응적 모멘트 추정) 을 사용하였습니다.
* metrics: 모형 훈련 과정에서 기록할 요소를 정합니다. 여기에서는 정확도'accuracy'를 사용하였습니다.

In [ ]:
history = model.fit(x_train.toarray(), y_train,
                    epochs=30, callbacks=[tf.keras.callbacks.EarlyStopping()],
                    validation_split=.3, verbose=0)

* x_train.toarray(): 훈련 데이터
* y_train: 지도 학습(supervised learning)에서 카테고리(label) 데이터
* epochs: 에포크 횟수. 30일 경우 전체 데이터를 30번 학습합니다.
* validation_split: 훈련(train)과 검증(validate) 데이터셋을 분리하는 비율. 여기서는 0.3 으로 설정하였고, 이는 1795개 중 30% 인 539개(반올림)를 검증용 데이터로 나눈다는 의미입니다.
* callbacks: tf.keras.callbacks.EarlyStopping() 옵션은 모형의 validation loss가 3개 에포크 동안 낮아지지 않을 경우 학습을 멈추게 됩니다.
* verbose: 모형 학습 중 출력되는 텍스트를 조절하는 옵션(verbosity). 0, 1, 2의 3개가 있으며verbose=0 일 경우, 아무런 문구도 출력되지 않습니다. verbose=2 일 경우, 진행상태를 나타내는 상태바(====)가 표시되지 않습니다.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train_loss', 'val_loss'])
plt.title('LOSS')
plt.show()

#### 가중치

In [ ]:
import pandas as pd

w, _ = model.weights
weights = pd.DataFrame(w.numpy())

weights.columns = ['motorcycle', 'baseball', 'hockey']
weights['word'] = tfidf.get_feature_names()

weights.sort_values('motorcycle', ascending=False).head()